# 椭圆轨道

In [3]:
import torch
import torchvision
import numpy as np
import importlib
from argparse import ArgumentParser, Namespace
from arguments import ModelParams, PipelineParams, OptimizationParams
from gaussian_renderer import render
from scene import Scene, GaussianModel
import tqdm as notebook_tqdm

In [25]:
parser = ArgumentParser(description="Training script parameters")
lp = ModelParams(parser)
op = OptimizationParams(parser)
pp = PipelineParams(parser)
parser.add_argument('--ip', type=str, default="127.0.0.1")
parser.add_argument('--port', type=int, default=6009)
parser.add_argument('--debug_from', type=int, default=-1)
parser.add_argument('--detect_anomaly', action='store_true', default=False)
parser.add_argument("--test_iterations", nargs="+", type=int, default=[7_000, 30_000])
parser.add_argument("--save_iterations", nargs="+", type=int, default=[7_000, 30_000])
parser.add_argument("--quiet", action="store_true")
parser.add_argument("--checkpoint_iterations", nargs="+", type=int, default=[7_000, 30_000])
parser.add_argument("--start_checkpoint", type=str, default = None)
args = parser.parse_args(["-s", "data/kitchen_8", "-m", "output/kitchen_8"])
dataset = lp.extract(args)
opt = op.extract(args)
pipe = pp.extract(args)

In [26]:
gaussians = GaussianModel(dataset.sh_degree, opt.optimizer_type)
scene = Scene(dataset, gaussians)

Reading camera 8/8
Loading Training Cameras
Loading Test Cameras
Number of points at initialisation :  22


In [27]:
views = scene.getTrainCameras()

In [28]:
poses = []
for view in views:
    pose = np.eye(4)
    pose[:3] = np.concatenate([view.R.T, view.T[:, None]], 1)
    pose = np.linalg.inv(pose)
    pose[:, 1:3] *= -1
    poses.append(pose)

# DUSt3R